In [ ]:
!pip install scikit-plot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scipy import interp
from itertools import cycle

In [ ]:
ds= pd.read_csv('/content/drive/MyDrive/Code/dataset/df_spark.csv')
ds = ds.drop(['Unnamed: 0'], axis=1)
ds.head()
ds.shape
# drop unnamed column

(357941, 12)

In [ ]:
#normalize
y = ds.iloc[:,0].values
X = ds.iloc[:,1:].values

sc = StandardScaler()
X = sc.fit_transform(X)

y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5, 6, 7])
n_classes = y.shape[1]
print(n_classes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# split data

8


In [ ]:
target_names = ['DoS', 'Probing',	'MaliciousControl', 'MaliciousOperation', 'Scan', 'Spying', 'WrongSetup', 'Normal']

In [ ]:
#decision tree with 5 fold cv
from sklearn import tree
pipe_tree = Pipeline([('clf', OneVsRestClassifier(tree.DecisionTreeClassifier(max_depth=3)))])
train_sizes_tree, train_scores_tree, test_scores_tree = learning_curve(estimator=pipe_tree,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_tree = np.mean(train_scores_tree, axis=1)
train_std_tree = np.std(train_scores_tree, axis=1)
test_mean_tree = np.mean(test_scores_tree, axis=1)
test_std_tree = np.std(test_scores_tree, axis=1)

In [ ]:
for i in train_mean_tree:
    print(i)

0.9812956172516152
0.9817548454688317
0.9852512353690974
0.9789777414975489
0.980469492093647


In [ ]:
for i in test_mean_tree:
    print(i)

0.9564565061019454
0.9564565061019457
0.9717691284071845
0.9716769355895909
0.9717244289195719


In [ ]:
#dt training
pipe_tree = pipe_tree.fit(X_train, y_train)
y_pred_test = pipe_tree.predict(X_test)
y_pred_train=pipe_tree.predict(X_train)
# print dt_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))


# print dt_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
#dt classification report
print(classification_report(y_train, y_pred_train, target_names=target_names))




Test_Accuracy: 0.9775384486443448
Test_Precision: 0.9780936925523914
Test_Recall: 0.9824693737864756
Test_f1-score: 0.9749142810076936
                    precision    recall  f1-score   support

               DoS       1.00      0.33      0.49      1168
           Probing       0.95      0.87      0.91        61
  MaliciousControl       1.00      0.98      0.99       179
MaliciousOperation       1.00      0.51      0.68       181
              Scan       1.00      0.02      0.05       299
            Spying       1.00      0.26      0.41       107
        WrongSetup       1.00      1.00      1.00        29
            Normal       0.98      1.00      0.99     69565

         micro avg       0.98      0.98      0.98     71589
         macro avg       0.99      0.62      0.69     71589
      weighted avg       0.98      0.98      0.97     71589
       samples avg       0.98      0.98      0.98     71589



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train_Accuracy: 0.9772797116835223
Train_Precision: 0.9778688265825421
Train_Recall: 0.9822735654020227
Train_f1-score: 0.9744304413078988
                    precision    recall  f1-score   support

               DoS       1.00      0.30      0.46      4612
           Probing       0.94      0.88      0.91       281
  MaliciousControl       1.00      0.98      0.99       710
MaliciousOperation       1.00      0.52      0.68       624
              Scan       1.00      0.01      0.02      1248
            Spying       1.00      0.34      0.51       425
        WrongSetup       1.00      1.00      1.00        93
            Normal       0.98      1.00      0.99    278359

         micro avg       0.98      0.98      0.98    286352
         macro avg       0.99      0.63      0.70    286352
      weighted avg       0.98      0.98      0.97    286352
       samples avg       0.98      0.98      0.98    286352



In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
pipe_rnd = Pipeline([('clf', OneVsRestClassifier(RandomForestClassifier(max_depth=3, n_estimators=100)))])
train_sizes_rnd, train_scores_rnd, test_scores_rnd = learning_curve(estimator=pipe_rnd,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_rnd = np.mean(train_scores_rnd, axis=1)
train_std_rnd = np.std(train_scores_rnd, axis=1)
test_mean_rnd = np.mean(test_scores_rnd, axis=1)
test_std_rnd = np.std(test_scores_rnd, axis=1)

In [ ]:
for i in train_mean_rnd:
    print(i)

0.9793225074209884
0.982332809498865
0.9852477431596347
0.9825109895626438
0.9831955076269765


In [ ]:
for i in test_mean_rnd:
    print(i)

0.963116840236158
0.963116840236158
0.9720149795891222
0.9718669107046667
0.9718417669819045


In [ ]:
#rf training
pipe_rnd = pipe_rnd.fit(X_train, y_train)
y_pred_test = pipe_rnd.predict(X_test)
y_pred_train=pipe_rnd.predict(X_train)
# print rf_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))

# print rf_train_scores of accuracy,precision,recall,f1-score, cohens kappa
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
print(classification_report(y_train, y_pred_train, target_names=target_names))




Test_Accuracy: 0.9775803545237397
Test_Precision: 0.9764072656966974
Test_Recall: 0.9812121974046292


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test_f1-score: 0.9735141896682264
                    precision    recall  f1-score   support

               DoS       1.00      0.34      0.51      1139
           Probing       1.00      0.28      0.43        65
  MaliciousControl       1.00      0.96      0.98       176
MaliciousOperation       1.00      0.42      0.60       172
              Scan       1.00      0.02      0.04       321
            Spying       0.00      0.00      0.00       124
        WrongSetup       1.00      1.00      1.00        32
            Normal       0.98      1.00      0.99     69560

         micro avg       0.98      0.98      0.98     71589
         macro avg       0.87      0.50      0.57     71589
      weighted avg       0.98      0.98      0.97     71589
       samples avg       0.98      0.98      0.98     71589



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train_Accuracy: 0.9773390791752807
Train_Precision: 0.9765071881801157
Train_Recall: 0.9812363803989496
Train_f1-score: 0.9732948457426471
                    precision    recall  f1-score   support

               DoS       1.00      0.30      0.46      4641
           Probing       1.00      0.30      0.46       277
  MaliciousControl       1.00      0.97      0.98       713
MaliciousOperation       1.00      0.55      0.71       633
              Scan       1.00      0.01      0.02      1226
            Spying       0.00      0.00      0.00       408
        WrongSetup       1.00      1.00      1.00        90
            Normal       0.98      1.00      0.99    278364

         micro avg       0.98      0.98      0.98    286352
         macro avg       0.87      0.51      0.58    286352
      weighted avg       0.98      0.98      0.97    286352
       samples avg       0.98      0.98      0.98    286352



In [ ]:
# svm
from sklearn.svm import SVC
pipe_svc = Pipeline([('clf', OneVsRestClassifier(SVC(C=1000)))])
train_sizes_svc, train_scores_svc, test_scores_svc = learning_curve(estimator=pipe_svc,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_svc = np.mean(train_scores_svc, axis=1)
train_std_svc = np.std(train_scores_svc, axis=1)
test_mean_svc = np.mean(test_scores_svc, axis=1)
test_std_svc = np.std(test_scores_svc, axis=1)


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
for i in test_mean_svc:
    print(i)

0.8919568692913069
0.8979355249709421
0.9729620589416047
0.973970608718969
0.9755295269449837


In [ ]:
for i in test_mean_svc:
    print(i)

0.8919568692913069
0.8979355249709421
0.9729620589416047
0.973970608718969
0.9755295269449837


In [ ]:
#svm training
pipe_svc = pipe_svc.fit(X_train, y_train)
y_pred_test = pipe_svc.predict(X_test)
y_pred_train=pipe_svc.predict(X_train)
# print svc_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))

# print svc_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
print(classification_report(y_train, y_pred_train, target_names=target_names))



Test_Accuracy: 0.9944265180404811
Test_Precision: 0.9943390833952498
Test_Recall: 0.9944265180404811
Test_f1-score: 0.9938895757770919
                    precision    recall  f1-score   support

               DoS       0.98      0.66      0.79      1117
           Probing       1.00      1.00      1.00        68
  MaliciousControl       1.00      1.00      1.00       156
MaliciousOperation       1.00      1.00      1.00       169
              Scan       1.00      1.00      1.00       303
            Spying       1.00      1.00      1.00       112
        WrongSetup       1.00      1.00      1.00        33
            Normal       0.99      1.00      1.00     69631

         micro avg       0.99      0.99      0.99     71589
         macro avg       1.00      0.96      0.97     71589
      weighted avg       0.99      0.99      0.99     71589
       samples avg       0.99      0.99      0.99     71589

Train_Accuracy: 0.9941645247806895
Train_Precision: 0.9941026230931067
Train_Recal